In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
print("--- DATA OVERVIEW ---")

df = pd.read_csv('../data/Intern_case_study_data.csv')

print(df.info())
print(df.head())

--- DATA OVERVIEW ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88076 entries, 0 to 88075
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   anon_user_id      88076 non-null  int64 
 1   order_date        88076 non-null  object
 2   payment_result    88076 non-null  object
 3   price_tier        88076 non-null  object
 4   dose_bucket       88076 non-null  object
 5   region            88076 non-null  object
 6   pharmacy_channel  88076 non-null  object
dtypes: int64(1), object(6)
memory usage: 4.7+ MB
None
          anon_user_id  order_date payment_result price_tier dose_bucket  \
0 -1000298151300261046  2025-04-06           paid        LOW       <=5.0   
1 -1000298151300261046  2025-04-25           paid        LOW       <=5.0   
2 -1000298151300261046  2025-08-20           paid        LOW       <=5.0   
3 -1000298151300261046  2025-09-12           paid        LOW       <=5.0   
4 -100029815130026104

In [12]:
print("--- MISSING/DUPLICATED VALUES ANALYSIS ---")
missing_summary = df.isnull().sum()
print(missing_summary)
print()

duplicates = df.duplicated()
print(f"Duplicated rows: {duplicates.sum()}\n") # this may refer to a customer purchasing multiple identical products at the same time

print(f"Number of unique customers: {df['anon_user_id'].nunique()}\n")
avg_transactions_per_customer = df.shape[0] / df['anon_user_id'].nunique()
max_transactions_by_customer = df['anon_user_id'].value_counts().max()
min_transactions_by_customer = df['anon_user_id'].value_counts().min()
print(f"Average transactions per customer: {avg_transactions_per_customer:.2f}")
print(f"Max transactions by a single customer: {max_transactions_by_customer}")
print(f"Min transactions by a single customer: {min_transactions_by_customer}")

--- MISSING/DUPLICATED VALUES ANALYSIS ---
anon_user_id        0
order_date          0
payment_result      0
price_tier          0
dose_bucket         0
region              0
pharmacy_channel    0
dtype: int64

Duplicated rows: 319

Number of unique customers: 20719

Average transactions per customer: 4.25
Max transactions by a single customer: 30
Min transactions by a single customer: 1


In [13]:
print("--- EXAMINING CATEGORICAL VARIABLES --")

categorical_cols = ['payment_result', 'price_tier', 'dose_bucket', 'region', 'pharmacy_channel']

for col in categorical_cols:
    print(f"{col}: {df[col].unique().tolist()}")

--- EXAMINING CATEGORICAL VARIABLES --
payment_result: ['paid', 'refunded']
price_tier: ['LOW', 'MID', 'HIGH']
dose_bucket: ['<=5.0', '5.0–10.0', '>10.0']
region: ['Ontario', 'Quebec', 'West', 'Other', 'Prairies']
pharmacy_channel: ['Partner Pharmacy', 'In-house Pharmacy']


In [14]:
print("--- DATA CLEANING --")
# Convert 'date' column to datetime
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d')


# Ordinal categorical columns to numerical codes
ordinal_mappings = {
    'price_tier': {'LOW': 1, 'MID': 2, 'HIGH': 3},
    'dose_bucket': {'<=5.0': 1, '5.0–10.0': 2, '>10.0': 3}
}
for col, mapping in ordinal_mappings.items():
    df[col] = df[col].map(mapping)

# Convert categorical columns to 'category' dtype 
categorical_cols = ['payment_result', 'price_tier', 'dose_bucket', 'region', 'pharmacy_channel']
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Boolean mapping
df['is_partner_pharmacy'] = np.where(df['pharmacy_channel'] == 'Partner Pharmacy', 1, 0)
df.drop(columns=['pharmacy_channel'], inplace=True)

df['is_paid'] = np.where(df['payment_result'] == 'paid', 1, 0)
df.drop(columns=['payment_result'], inplace=True)

print(df.head())

--- DATA CLEANING --
          anon_user_id order_date price_tier dose_bucket   region  \
0 -1000298151300261046 2025-04-06          1           1  Ontario   
1 -1000298151300261046 2025-04-25          1           1  Ontario   
2 -1000298151300261046 2025-08-20          1           1  Ontario   
3 -1000298151300261046 2025-09-12          1           1  Ontario   
4 -1000298151300261046 2025-11-01          1           1  Ontario   

   is_partner_pharmacy  is_paid  
0                    1        1  
1                    1        1  
2                    1        1  
3                    1        1  
4                    1        1  


In [15]:
print("--- DATE RANGE ANALYSIS ---")

print(f"Earliest order: {df['order_date'].min()}")
print(f"Latest order: {df['order_date'].max()}")

--- DATE RANGE ANALYSIS ---
Earliest order: 2025-01-10 00:00:00
Latest order: 2026-01-29 00:00:00


Although there exists entries that are future dated (as of Jan 24th, 2026), I will assume that all entries are valid and should be included in the analysis.

In [20]:
print("--- PAYMENT SUCCESS RATE ANALYSIS ---")
total_orders = df.shape[0]
successful_payments = df['is_paid'].sum()
payment_success_rate = (successful_payments / total_orders) * 100
print(f"Payment Success Rate: {payment_success_rate:.2f}%")

# removing unsuccessful payments for further analysis
df = df[df['is_paid'] == 1]
df.drop(columns=['is_paid'], inplace=True)

--- PAYMENT SUCCESS RATE ANALYSIS ---
Payment Success Rate: 100.00%


In [21]:
output_file = '../data/cleaned_data.csv'
df.to_csv(output_file, index=False)